In [1]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract
from collections import defaultdict

<IPython.core.display.Javascript object>

In [2]:
keyList = ["Name",
            "Details"]

In [4]:
genres = []
file = open('genres.txt', "r")
lines = file.readlines()
lines = list(set(lines))

count = 0
for line in lines:
    genre = {key: None for key in keyList}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    genre_info = line.strip()
    header = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    response = requests.get(genre_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('genre.html', 'w', encoding="utf-8") as f:
        f.write(page_contents)

    # read it html page to scrape information
    with open('genre.html', 'r', encoding="utf-8") as f:
        html_source = f.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_genre = BeautifulSoup(html_source, 'html.parser')

    name = doc_genre.find('div', class_='genreHeader')
    genre['Name']= name.text.strip()

    details = doc_genre.find('div', class_='mediumText reviewText')
    if details != None:
        for br in details.select("br"):
            br.replace_with("\n")
        content = details.find('span', attrs={'style':'display:none'})
        if content is None:
            genre['Details'] = details.text.strip()
        else:
            genre['Details'] = content.text.strip()

    print(genre)
    genres.append(genre)


Line1: https://www.goodreads.com/genres/school
{'Name': 'School', 'Details': 'See also school stories.'}
Line2: https://www.goodreads.com/genres/epic-fantasy
{'Name': 'Epic Fantasy', 'Details': "Epic Fantasy is a subgenre of fantasy that is set in invented or parallel world. It is most commonly considered to be the same genre as High Fantasy, since they share many of the same elements, but Epic Fantasy takes the story even farther, in a bigger and more epic way (e.g. the stakes are greater; the land/world is in peril; a great evil threatens the world; the conflict shakes all who dwell in the world of the story).\n\nEpic Fantasy became more famous through the works of such as: Brandon Sanderson, Brent Weeks, Robert Jordan, Robin Hobb, Patrick Rothfuss, Scott Lynch, Joe Abercrombie, Terry Brooks, Raymond E. Feist, Tad Williams, George R.R. Martin, Steven Erikson, and more; though it's origins come long before, from J.R.R. Tolkien's The Lord of the Rings."}
Line3: https://www.goodreads.co

In [5]:
genres_df = pd.DataFrame(data=genres)
genres_df

,Name,Details
0,School,See also school stories.
1,Epic Fantasy,Epic Fantasy is a subgenre of fantasy that is ...
2,Mythology,The term mythology can refer to a body of myth...
3,Middle Grade,Middle grade is fiction aimed at children aged...
4,Self Help,"Self-help, or self-improvement, is a self-guid..."
...,...,...
57,Adult,Fiction intended for adults.
58,World War II,The global conflict that took place between 19...
59,Urban Fantasy,Urban fantasy is a subgenre of fantasy in whic...
60,Teen,Books for teenagers.


In [6]:
genres_df.to_csv('genres.csv', index = True)